## Mel Spectrogram Generation

This notebook illustrates an example to generate a mel spectrogram from an audio file.

This example uses:
- `librosa` library for audio I/O and processing
- `numpy` for some numerical processing
- `matplotlib` for plotting and saving the spectrogram as an image file

Variables:
- audio_path: path to audio file
- image_path: path to save resulting spectrogram image to. Most image types are supported.
- resample_rate: You may choose to resample the audio at a particular frequency.
  Leave as `None` if you do not wish to resample, however this may result in a noisy spectrogram.
- Please refer to the [librosa documentation](https://librosa.org/doc/latest/generated/librosa.feature.melspectrogram.html) for an explanation of the following parameters:
    - n_mels
    - hop_length
    - n_fft
    - window
- image_width: width of output image in inches
- image_height: height of output image in inches
- plot_without_axis_or_labels: setting to remove graph axis and labels in the output image

In [ ]:
# # Imports
import librosa
from librosa import display as librosa_display
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# # Variables
audio_path = "./example/input/example_heart_beat.wav"
image_path = "./example/output/example_heart_beat.png"

resample_rate = 22050
n_mels = 128
hop_length = 512
n_fft = hop_length * 2
window = np.hanning(hop_length * 2)
image_width = 18.5
image_height = 10.5

plot_without_axis_or_labels = True

In [ ]:
# # Audio input and processing
# load audio
wave, rate = librosa.load(audio_path, sr=resample_rate)

# trim audio of leading and trailing silence
trim, _ = librosa.effects.trim(wave)

# convert to mel spectrogram
mel = librosa.feature.melspectrogram(
    y=trim,
    sr=rate,
    n_mels=n_mels,
    n_fft=hop_length * 2,
    hop_length=hop_length,
    window=window,
)

# convert scale from amplitude squared to decibels (relative to max)
db = librosa.power_to_db(mel, ref=np.max)

In [ ]:
# # Spectrogram plotting
fig, ax = plt.subplots(figsize=(image_width, image_height))
img = librosa_display.specshow(db, x_axis="s", y_axis="log", sr=rate)

if plot_without_axis_or_labels:
    fig.patch.set_visible(False)
    ax.axis("off")
else:
    fig.colorbar(img, ax=ax, format="%+2.f dB")

plt.show()

In [ ]:
# # Saving image
if plot_without_axis_or_labels:
    transparent = True
    pad_inches = 0.0
else:
    transparent = False
    pad_inches = 0.1

fig.savefig(
    image_path, bbox_inches="tight", transparent=transparent, pad_inches=pad_inches
)

## Bonus tip
Instead of saving the spectogram as an image file, you may instead convert the plot to a numpy array for further processing.
For example, you can combine this recipe with the [audio_to_video](../audio_to_mel_spectrogram) recipe by passing the rgba array to `mpy.ImageClip` instead of an image file.


In [ ]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

canvas = FigureCanvas(fig)
canvas.draw()
img = np.asarray(canvas.buffer_rgba())
# Now, do further processing with img...
